In [2]:
#필요한 library 들을 load 합니다.
import os 

import random
import numpy as np
from tqdm import tqdm

import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import torchvision.transforms as transforms

from multiprocessing import cpu_count
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torch.nn import CrossEntropyLoss
from torchvision.models import efficientnet_v2_s
from torchvision.models import efficientnet_b0
from torchvision.models.efficientnet import EfficientNet_V2_S_Weights
from torchvision.models import EfficientNet_B0_Weights
from sklearn.model_selection import train_test_split

from PIL import Image

In [3]:
USE_CUDA = torch.cuda.is_available() # GPU를 사용가능하면 True, 아니라면 False를 리턴
device = torch.device("cuda" if USE_CUDA else "cpu") # GPU 사용 가능하면 사용하고 아니면 CPU 사용
print("다음 기기로 학습합니다:", device)

# from google.colab import drive
# drive.mount('/content/jdrive/')

다음 기기로 학습합니다: cuda


In [4]:
%cd /data/

[Errno 2] No such file or directory: '/data/'
/home/jupyter-k7a506


In [5]:
!rm -rf /content/image

In [6]:
!rm -rf /content/__MACOSX

In [34]:
!unzip -qq /content/jdrive/MyDrive/Colab\ Notebooks/image.zip


In [10]:
# !unzip image2.zip

Archive:  image2.zip
   creating: image2/
   creating: image2/anger/
  inflating: image2/anger/anger1.jpg  
 extracting: image2/anger/anger10.jpg  
  inflating: image2/anger/anger100.jpg  
  inflating: image2/anger/anger101.jpg  
  inflating: image2/anger/anger102.jpg  
  inflating: image2/anger/anger103.jpg  
  inflating: image2/anger/anger104.jpg  
  inflating: image2/anger/anger105.jpg  
  inflating: image2/anger/anger106.jpg  
  inflating: image2/anger/anger107.jpg  
  inflating: image2/anger/anger108.jpg  
  inflating: image2/anger/anger109.jpg  
  inflating: image2/anger/anger11.jpg  
  inflating: image2/anger/anger110.jpg  
  inflating: image2/anger/anger111.jpg  
  inflating: image2/anger/anger112.jpg  
  inflating: image2/anger/anger113.jpg  
  inflating: image2/anger/anger114.jpg  
  inflating: image2/anger/anger115.jpg  
  inflating: image2/anger/anger116.jpg  
  inflating: image2/anger/anger117.jpg  
  inflating: image2/anger/anger118.jpg  
  inflating: image2/anger/anger11

In [22]:
transform = transforms.Compose([
    transforms.ToTensor(), #이미지 데이터를 tensor 데이터 포멧으로 바꾸어줍니다.
    transforms.Resize([224,224]), #이미지의 크기가 다를 수 있으니 크기를 통일해 줍니다.
    
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.Normalize(mean=(0.8512447, 0.46590397, 0.21504538), std=(0.85379875, 0.96438026, 0.6715451)) # 이미지들의 픽셀 r, g, b의 평균 => 아래 코드에 구하는 부분있으니 참고할 것
])

In [23]:
train_data = datasets.ImageFolder(root='./test_img/',transform=transform)
train_data

Dataset ImageFolder
    Number of datapoints: 192
    Root location: ./test_img/
    StandardTransform
Transform: Compose(
               ToTensor()
               Resize(size=[224, 224], interpolation=bilinear, max_size=None, antialias=None)
               RandomHorizontalFlip(p=0.5)
               RandomVerticalFlip(p=0.5)
               Normalize(mean=(0.8512447, 0.46590397, 0.21504538), std=(0.85379875, 0.96438026, 0.6715451))
           )

In [24]:
train_idx, valid_idx = train_test_split(np.arange(len(train_data)), test_size=0.2, train_size = 0.8,
                                        random_state=42, shuffle=True, stratify=train_data.targets)

In [34]:
batch_size = 32 #batchsize는 2의배수롤 설정하시면 되는데 데이터 크기가 클수록 batch_size도 키워주시면 됩니다. 보통 32,64 사용하시면됩니다. 이 이상키우시면 cuda가 메모리양을 감당못하고 꺼질수있습니다.
num_workers = 1

In [35]:
train_loader = DataLoader(train_data, batch_size=batch_size, 
                          sampler=SubsetRandomSampler(train_idx), num_workers=num_workers)
valid_loader = DataLoader(train_data, batch_size=batch_size, 
                          sampler=SubsetRandomSampler(valid_idx), num_workers=num_workers)

In [36]:
train_total = len(train_idx)
valid_total = len(valid_idx)

train_batches = len(train_loader)
valid_batches = len(valid_loader)

In [37]:
print('total train imgs :',train_total,'/ total train batches :', train_batches)
print('total valid imgs :',valid_total, '/ total valid batches :', valid_batches)

total train imgs : 153 / total train batches : 5
total valid imgs : 39 / total valid batches : 2


In [38]:
# import numpy as np

# meanRGB = [np.mean(x.numpy(), axis=(1,2)) for x,_ in train_loader]
# stdRGB = [np.std(x.numpy(), axis=(1,2)) for x,_ in train_loader]

# meanR = np.mean([m[0] for m in meanRGB])
# meanG = np.mean([m[1] for m in meanRGB])
# meanB = np.mean([m[2] for m in meanRGB])

# stdR = np.mean([s[0] for s in stdRGB])
# stdG = np.mean([s[1] for s in stdRGB])
# stdB = np.mean([s[2] for s in stdRGB])

# print(meanR, meanG, meanB)
# print(stdR, stdG, stdB)

In [39]:
class PretrainModel(nn.Module):
    def __init__(self, pretrained):
        super().__init__()
        self.pretrained = pretrained
        self.FC = nn.Linear(1000,6)
    
    def forward(self,x):
        x = self.pretrained(x)
        x = self.FC(x)
        return x

In [40]:
weights = EfficientNet_V2_S_Weights.IMAGENET1K_V1
pretrained = efficientnet_v2_s(weights = weights)

In [41]:
net = PretrainModel(pretrained).to(device)

In [42]:
criterion = CrossEntropyLoss().to(device)
#criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = optim.AdamW(params=net.parameters(), lr=0.001)
#optimizer = torch.optim.SGD(params=net.parameters(), lr = 0.001)
epochs = 20

In [43]:
#pretrain model
for epoch in range(epochs):
    net.train()
    
    train_loss = 0 
    train_correct = 0
    
    tqdm_dataset = tqdm(train_loader)
    for x,y in tqdm_dataset:
        x = x.to(device)
        y = y.to(device)
        outputs = net(x)
        loss = criterion(outputs,y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        train_correct += predicted.eq(y).sum().item()
        
        tqdm_dataset.set_postfix({
            'Epoch': epoch + 1,
            'Loss': '{:06f}'.format(loss.item()),
        })

    train_loss = train_loss / train_batches
    train_acc = train_correct / train_total
    
    net.eval()
    
    valid_loss = 0 
    valid_correct = 0
    tqdm_dataset = tqdm(valid_loader)
    with torch.no_grad():
        for x,y in tqdm_dataset:
            x = x.to(device)
            y = y.to(device)
            
            outputs = net(x)
            loss = criterion(outputs, y)
            valid_loss += loss.item()
            _, predicted = outputs.max(1)
            valid_correct += predicted.eq(y).sum().item()
            
            tqdm_dataset.set_postfix({
                'Epoch': epoch + 1,
                'Loss': '{:06f}'.format(loss.item()),
            })
            
    valid_loss = valid_loss / valid_batches
    valid_acc = valid_correct / valid_total
        
    print('epochs',epoch+1, 'train loss',train_loss,'train acc', train_acc, 'valid loss',valid_loss, 'valid acc',valid_acc)



100%|██████████| 2/2 [00:01<00:00,  1.01it/s, Epoch=1, Loss=1.017970]


epochs 1 train loss 1.2854759216308593 train acc 0.4117647058823529 valid loss 0.9444306492805481 valid acc 0.5641025641025641


100%|██████████| 2/2 [00:01<00:00,  1.12it/s, Epoch=2, Loss=1.683912]


epochs 2 train loss 0.530923992395401 train acc 0.7908496732026143 valid loss 1.8929298520088196 valid acc 0.46153846153846156


100%|██████████| 2/2 [00:01<00:00,  1.12it/s, Epoch=3, Loss=1.887602]


...

epochs 18 train loss 0.1954023594968021 train acc 0.9673202614379085 valid loss 2.0845059752464294 valid acc 0.46153846153846156


100%|██████████| 2/2 [00:02<00:00,  1.25s/it, Epoch=19, Loss=0.997355]


epochs 19 train loss 0.135960038844496 train acc 0.9607843137254902 valid loss 1.6461076438426971 valid acc 0.5128205128205128


100%|██████████| 2/2 [00:02<00:00,  1.14s/it, Epoch=20, Loss=3.291861]

epochs 20 train loss 0.1597859959118068 train acc 0.954248366013072 valid loss 3.1056745052337646 valid acc 0.41025641025641024


In [44]:
def plot_training(history):
    acc = history['acc']
    val_acc = history['val_acc']
    loss = history['loss']
    val_loss = history['val_loss']
    epochs = range(len(acc))

    fig, axes = plt.subplots(1, 2, figsize=(15,5))
    
    axes[0].plot(epochs, acc, 'r-', label='Training Accuracy')
    axes[0].plot(epochs, val_acc, 'b--', label='Validation Accuracy')
    axes[0].set_title('Training and Validation Accuracy')
    axes[0].legend(loc='best')

    axes[1].plot(epochs, loss, 'r-', label='Training Loss')
    axes[1].plot(epochs, val_loss, 'b--', label='Validation Loss')
    axes[1].set_title('Training and Validation Loss')
    axes[1].legend(loc='best')
    
    plt.show()
    
plot_training(history)

NameError: name 'history' is not defined

In [17]:
for layer in net.layers:
    layer.trainable = False

for layer in net.layers[:50]:
    layer.trainable = True

AttributeError: 'PretrainModel' object has no attribute 'layers'

In [15]:
torch.save(net, 'model_batch128.pt')

In [16]:
torch.save(net.state_dict(), 'model_batch128_state_dict.pt')